# Opdracht 3 Soort van YOLO

YOLO
je hebt een grid en bepaald wat is de kans dat dit celletje de centrale cel is van bijvoorbeedl een plaatsje van een hond. Het model doet een voorspelling. YOLO is een pytorch ding. we gebruiken het model dat al gebruikt is om images te detecteren. Kijken waar er bounding boxes zijn. missende data, de totale oppervlak van het plaatjem hoeveel wordt opgevuld door tegels?
Oppervlak keer 250? en dan krijg je de opvulling, check of dit correct is. per plaatje breedte en hoogte bepalen. Per afbeelding een bestand maken voor de labels directory. Coords-idc.txt is een plaatje en de bounding box van tumoren. linksboven naar rechtsonder. 

### Laad libraries

In [1]:
import os
import pandas as pd
import shutil
import random

### Data omzetten naar YOLO-formaat

In [2]:
# Functie om annotaties om te zetten naar YOLO-formaat
def convert_to_yolo_format(coords_file, images_folder, output_folder):
    """
    Converteert annotaties van een CSV-bestand naar YOLO-formaat.

    Parameters:
    - coords_file (str): Pad naar het CSV-bestand met annotaties.
    - images_folder (str): Map met afbeeldingsbestanden.
    - output_folder (str): Map om de YOLO-annotaties op te slaan.

    Returns:
    None
    """
    # Maak output map aan als die nog niet bestaat
    os.makedirs(output_folder, exist_ok=True)

    # Laad annotaties
    coords = pd.read_csv(coords_file, header=None, names=["image_id", "xmin", "ymin", "xmax", "ymax"])

    for image_id in coords["image_id"].unique():
        # Selecteer alle annotaties voor een afbeelding
        image_coords = coords[coords["image_id"] == image_id]
        
        # Bereken de afmetingen van de afbeelding
        image_path = os.path.join(images_folder, f"{image_id}.jpeg")
        if not os.path.exists(image_path):
            print(f"Afbeelding {image_path} niet gevonden.")
            continue
        
        from PIL import Image
        with Image.open(image_path) as img:
            width, height = img.size

        # Genereer YOLO-annotaties
        yolo_annotations = []
        for _, row in image_coords.iterrows():
            x_center = ((row["xmin"] + row["xmax"]) / 2) / width
            y_center = ((row["ymin"] + row["ymax"]) / 2) / height
            bbox_width = (row["xmax"] - row["xmin"]) / width
            bbox_height = (row["ymax"] - row["ymin"]) / height
            yolo_annotations.append(f"1 {x_center} {y_center} {bbox_width} {bbox_height}")

        # Schrijf annotaties naar bestand
        annotation_path = os.path.join(output_folder, f"{image_id}.txt")
        with open(annotation_path, "w") as f:
            f.write("\n".join(yolo_annotations))

# Bestandspaden
coords_file = "../data/coords-idc.txt"
images_folder = "../data/complete_images"
output_folder = "../data/yolo_labels"
convert_to_yolo_format(coords_file, images_folder, output_folder)

### Directory-structuur creëren

In [3]:
def create_dataset_structure(images_folder, labels_folder, output_folder, train_ratio=0.8):
    """
    Creëert een gestructureerde dataset met een training- en validatieset.

    Parameters:
    - images_folder (str): Map met afbeeldingsbestanden.
    - labels_folder (str): Map met YOLO-annotaties.
    - output_folder (str): Doelmap voor gestructureerde dataset.
    - train_ratio (float): Ratio van training-data tot validatie-data.

    Returns:
    None
    """
    # Maak mapstructuur
    for split in ["train", "val"]:
        os.makedirs(os.path.join(output_folder, "images", split), exist_ok=True)
        os.makedirs(os.path.join(output_folder, "labels", split), exist_ok=True)

    # Verdeel data in training- en validatiesets
    image_files = [f for f in os.listdir(images_folder) if f.endswith(".jpeg")]
    random.shuffle(image_files)
    train_split = int(len(image_files) * train_ratio)

    splits = {
        "train": image_files[:train_split],
        "val": image_files[train_split:]
    }

    for split, files in splits.items():
        for image_file in files:
            # Verplaats afbeelding
            src_image = os.path.join(images_folder, image_file)
            dest_image = os.path.join(output_folder, "images", split, image_file)
            shutil.copy(src_image, dest_image)

            # Verplaats corresponderend label
            label_file = image_file.replace(".jpeg", ".txt")
            src_label = os.path.join(labels_folder, label_file)
            dest_label = os.path.join(output_folder, "labels", split, label_file)
            if os.path.exists(src_label):
                shutil.copy(src_label, dest_label)

images_folder = "../data/complete_images"
labels_folder = "../data/yolo_labels"
output_folder = "../data/dataset"
create_dataset_structure(images_folder, labels_folder, output_folder)

### YOLO-configuratiebestand

Maak een configuratie-bestand dataset.yaml met daarin de volgende gegevens

train: /path/to/dataset/images/train

val: /path/to/dataset/images/val

nc: 1  # aantal klassen

names: ['tumor']  # lijst van klasse-namen

### Model trainen

In [4]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="dataset.yaml", epochs=3, single_cls=True, amp=False, verbose=True)

New https://pypi.org/project/ultralytics/8.3.52 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.51  Python-3.12.7 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train28, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sav

train: Scanning C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\labels\train.cache... 279 images, 0 backgrounds, 1 corrupt: 100%|██████████| 279/279 [00:00<?, ?it/s]

train: WARNING  C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\images\train\12906.jpeg: ignoring corrupt image/label: negative label values [  -0.014904    -0.35914   -0.036478    -0.22465    -0.38828]



val: Scanning C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\labels\val... 198 images, 0 backgrounds, 0 corrupt: 100%|██████████| 198/198 [00:00<00:00, 576.89it/s]


val: New cache created: C:\Users\isabe\Documents\Hanze\jaar_3_new\kw2\modelleren_van_kanker\casus_d\data\dataset\labels\val.cache
Plotting labels to runs\detect\train28\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train28
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        198        523      0.339      0.103      0.096     0.0342



3 epochs completed in 0.010 hours.
Optimizer stripped from runs\detect\train28\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train28\weights\best.pt, 6.2MB

Validating runs\detect\train28\weights\best.pt...
Ultralytics 8.3.51  Python-3.12.7 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.58it/s]


                   all        198        523      0.345      0.101     0.0961     0.0343
Speed: 0.3ms preprocess, 3.8ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs\detect\train28


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000018F8BA2B9B0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

### Voorspellingen doen

In [5]:
def make_predictions(model_path, test_images_folder, output_folder):
    pass